In [1]:
# Imports
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.formula.api as smf

In [2]:
# List all sheets in Excel file
file_path = 'data/PWT_EC1B1_2025.xlsx'
sheets = pd.ExcelFile(file_path).sheet_names
print(sheets)

['Info', 'Legend', 'rgdpna', 'rnna', 'emp', 'hc', 'pop', 'labsh']


In [3]:
# Combinde df of selected sheets
selected_sheets = sheets[2:]
merged_df = None

for sheet in selected_sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)
    
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on=['countrycode', 'country', 'currency_unit', 'year'], how='outer')

print(merged_df.head())

  countrycode country currency_unit  year        rgdpna          rnna  \
0         AGO  Angola        Kwanza  1970  54237.054688  295517.62500   
1         AGO  Angola        Kwanza  1971  57491.277344  314195.09375   
2         AGO  Angola        Kwanza  1972  57606.261719  332435.84375   
3         AGO  Angola        Kwanza  1973  62272.367188  352647.90625   
4         AGO  Angola        Kwanza  1974  64202.808594  373267.71875   

        emp        hc       pop     labsh  
0  3.666207  1.015686  5.890365  0.284385  
1  3.742484  1.018196  6.040777  0.284385  
2  3.853271  1.020712  6.248552  0.284385  
3  3.987807  1.023234  6.496962  0.284385  
4  4.130696  1.025762  6.761380  0.284385  


### **Data Cleaning**

In [4]:
# Drop countrycode, pop, labsh
df = merged_df.drop(columns=['countrycode','pop', 'labsh'])

In [5]:
# Drop empty emp 
df = df.dropna(subset=['emp'])

In [6]:
# Check any more na values
print(df[df.isna().any(axis=1)])

Empty DataFrame
Columns: [country, currency_unit, year, rgdpna, rnna, emp, hc]
Index: []


In [7]:
# Rename
df = df.rename(columns={'country': 'Country', 'currency_unit': 'Currency Unit', 'year': 'Year', 'rgdpna': 'Real GDP', 'rnna': 'Capital', 'emp': 'Labour', 'hc': 'Human Capital'})

In [8]:
# I am looking at Australia
aus_df = df[df['Country'] == 'Australia'].reset_index(drop=True).drop(columns=['Country', 'Currency Unit'])

### **Data Visualisation**

In [9]:
# Real GDP over time
fig = px.scatter(aus_df, x='Year', y='Real GDP', 
                 title = 'Real GDP Over Time (AUS)', 
                 labels={'Year': 'Year', 'Real_GDP': 'Real GDP'}, 
                 trendline='ols',
                 color_discrete_sequence=['#4682B4'])

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(family='sans-serif', size=20, color='black', weight='bold')
)

fig.update_traces(line=dict(color='#D3D3D3'), selector=dict(mode='lines'))

fig.show()

In [10]:
# Real GDP per capita over time
aus_df.loc[:, 'Real GDP per Capita'] = aus_df['Real GDP'] / aus_df['Labour']

fig = px.scatter(aus_df, x='Year', y='Real GDP per Capita', 
                 title = 'Real GDP per Capita Over Time (AUS)', 
                 labels={'Year': 'Year', 'Real GDP per Capita': 'Real GDP per Capita'}, 
                 trendline='ols',
                 color_discrete_sequence=['#4682B4'])

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(family='sans-serif', size=20, color='black', weight='bold')
)

fig.update_traces(line=dict(color='#D3D3D3'), selector=dict(mode='lines'))

fig.show()

In [11]:
# Real GDP over time (Euro)
euro_df = df[df['Currency Unit'] == 'Euro']


# Real GDP over time
fig = px.scatter(euro_df, x='Year', y='Real GDP', 
                 title = 'Real GDP Over Time (EUR)', 
                 labels={'Year': 'Year', 'Real GDP': 'Real GDP'}, 
                 trendline='ols',
                 color = 'Country',
                 color_continuous_scale='Blues')

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(family='sans-serif', size=20, color='black', weight='bold')
)

fig.update_traces(line=dict(color='#D3D3D3'), selector=dict(mode='lines'))

fig.show()

In [12]:
# Real GDP per capita over time
euro_df.loc[:, 'Real GDP per Capita'] = euro_df['Real GDP'] / euro_df['Labour']

fig = px.scatter(euro_df, x='Year', y='Real GDP per Capita', 
                 title = 'Real GDP per Capita Over Time (EUR)', 
                 labels={'Year': 'Year', 'Real GDP per Capita': 'Real GDP per Capita'}, 
                 trendline='ols',
                 color = 'Country',
                 color_continuous_scale='Blues')

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(family='sans-serif', size=20, color='black', weight='bold')
)

fig.update_traces(line=dict(color='#D3D3D3'), selector=dict(mode='lines'))
fig.update_traces(marker=dict(opacity=0.75))

fig.show()

/var/folders/3m/wqvcp0vs4n3fc_ywqy10vbhw0000gn/T/ipykernel_65661/3345772107.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### **Regressions**

In [13]:
# Group every 5 rows and calculate the average
df.loc[:, 'Real GDP per Capita'] = df['Real GDP'] / df['Labour']
grouped_df = df.groupby(df.index // 5).agg({'Country': 'first', 'Currency Unit': 'first', 'Year': 'first', 'Real GDP': 'mean', 'Capital': 'mean', 'Labour': 'mean', 'Human Capital': 'mean', 'Real GDP per Capita': 'mean'})

# Update the Year column to represent the grouped years
grouped_df['Year'] = grouped_df['Year'].astype(str) + '-' + (grouped_df['Year'] + 4).astype(str)

# Reset the index
grouped_df.reset_index(drop=True, inplace=True)

In [14]:
grouped_df['log_Real_GDP'] = np.log(grouped_df['Real GDP'])
grouped_df['log_Capital'] = np.log(grouped_df['Capital']) 
grouped_df['log_Labour'] = np.log(grouped_df['Labour']) 
grouped_df['log_Human Capital'] = np.log(grouped_df['Human Capital'])

In [15]:
formula = 'log_Real_GDP ~ log_Capital + log_Labour - 1'
model = smf.ols(formula=formula, data=grouped_df).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:           log_Real_GDP   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          4.261e+05
Date:                Fri, 21 Feb 2025   Prob (F-statistic):                        0.00
Time:                        17:27:36   Log-Likelihood:                         -1061.5
No. Observations:                1544   AIC:                                      2127.
Df Residuals:                    1542   BIC:                                      2138.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

**α = 0.8793**  
**β = 0.1501**

In [16]:
grouped_df['Predicted_log_Real_GDP'] = model.predict(grouped_df[['log_Capital', 'log_Labour']])
grouped_df['Predicted_Real_GDP'] = np.exp(grouped_df['Predicted_log_Real_GDP'])

In [17]:
# I am looking at Australia
aus_grouped_df = grouped_df[grouped_df['Country'] == 'Australia'].reset_index(drop=True).drop(columns=['Country', 'Currency Unit'])

In [18]:
# Actual Real GDP vs Predicted Real GDP 
import plotly.graph_objs as go

# Scatter plot for Real GDP
scatter_real_gdp = go.Scatter(
    x=aus_grouped_df['Year'],
    y=aus_grouped_df['Real GDP'],
    mode='markers',
    name='Real GDP',
    marker=dict(color='#4682B4')
)

# Scatter plot for Predicted Real GDP
scatter_predicted_gdp = go.Scatter(
    x=aus_grouped_df['Year'],
    y=aus_grouped_df['Predicted_Real_GDP'],
    mode='markers',
    name='Predicted Real GDP',
    marker=dict(color='#ADD8E6', opacity=0.75)
)

# Create layout
layout = go.Layout(
    title='Real GDP vs Predicted Real GDP',
    xaxis=dict(title='Year', tickformat='%Y'),
    yaxis=dict(title='GDP')
)

# Create figure and add scatter plots
fig = go.Figure(data=[scatter_real_gdp, scatter_predicted_gdp], layout=layout)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(family='sans-serif', size=20, color='black', weight='bold')
)

# Show the figure
fig.show()

Quite accurate actually! Becomes more accurate recently

In [21]:
# I am looking at Zimbabwe
zim_grouped_df = grouped_df[grouped_df['Country'] == 'Zimbabwe'].reset_index(drop=True).drop(columns=['Country', 'Currency Unit'])

In [22]:
# Actual Real GDP vs Predicted Real GDP 
import plotly.graph_objs as go

# Scatter plot for Real GDP
scatter_real_gdp = go.Scatter(
    x=zim_grouped_df['Year'],
    y=zim_grouped_df['Real GDP'],
    mode='markers',
    name='Real GDP',
    marker=dict(color='#4682B4')
)

# Scatter plot for Predicted Real GDP
scatter_predicted_gdp = go.Scatter(
    x=zim_grouped_df['Year'],
    y=zim_grouped_df['Predicted_Real_GDP'],
    mode='markers',
    name='Predicted Real GDP',
    marker=dict(color='#ADD8E6', opacity=0.75)
)

# Create layout
layout = go.Layout(
    title='Real GDP vs Predicted Real GDP',
    xaxis=dict(title='Year', tickformat='%Y'),
    yaxis=dict(title='GDP')
)

# Create figure and add scatter plots
fig = go.Figure(data=[scatter_real_gdp, scatter_predicted_gdp], layout=layout)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(family='sans-serif', size=20, color='black', weight='bold')
)

# Show the figure
fig.show()

Much less accurate!